In [1]:
#import required libraries
from pyspark import SparkContext

In [2]:
#To separate position and value
def map_func(x):
    temp = x.split(',')
    if(temp[0] == 'A'):
        for k in range(p):
            key_string = str(temp[1]+','+str(k))
            value_string = str('A,'+temp[2]+','+temp[3])
            yield(key_string,value_string) 
    else:
        for i in range(m):
            key_string = str(str(i)+','+temp[2])
            value_string = str('B,'+temp[1]+','+temp[3])
            yield(key_string,value_string) 

#To perform the sum of product operation             
def map_func2(x):
    temp_string = x.split('#')
    a = []
    b = []
    for x in range(n):
        a.append(0)
        b.append(0)

    for string in temp_string:
        temp = string.split(',')
        print(temp)
        if(temp[0]=='A'):
            a[int(temp[1])] = int(temp[2])
        else:
            b[int(temp[1])] = int(temp[2])
    ans = 0
    for i in range(n):
        ans += a[i] * b[i]
    return str(ans)            

SparkContext is the main entry point for Spark functionality. A SparkContext represents the connection to a Spark cluster, and can be used to create RDDs, accumulators and broadcast variables on that cluster. Once the context is created, a map transformation is applied on the input file. A map is a transformation operation. It applies to each element of RDD and it returns the result as new RDD. FlatMap allows returning 0, 1 or more elements from map function

In [3]:
#define matrix dimensions where A is mxn and B is nxp
m = 2
n = 2
p = 2

sc = SparkContext.getOrCreate();
numbers = sc.textFile("matrix_1.txt").flatMap(lambda line:line.split("\n"))
numbers.collect()

['A,0,0,1',
 'A,0,1,2',
 'A,1,0,3',
 'A,1,1,4',
 'B,0,0,1',
 'B,0,1,0',
 'B,1,0,0',
 'B,1,1,1']

Now, the position and the value are separated using map_func (defined above). This function generates the values required for multiplication. The output is passed through flatMap(). The output is of the form ('i,j', 'matrixName,index,value'), where (i,j) represents the position of the value in the matrix (matrixName) and index denotes the row/column value for matching the multiplication so that we can do: a_0 * b_0 + a_1 * b_1.

In [51]:
mappings = numbers.flatMap(map_func)
mappings.collect()

[('0,0', 'A,0,1'),
 ('0,1', 'A,0,1'),
 ('0,0', 'A,1,2'),
 ('0,1', 'A,1,2'),
 ('1,0', 'A,0,3'),
 ('1,1', 'A,0,3'),
 ('1,0', 'A,1,4'),
 ('1,1', 'A,1,4'),
 ('0,0', 'B,0,1'),
 ('1,0', 'B,0,1'),
 ('0,1', 'B,0,0'),
 ('1,1', 'B,0,0'),
 ('0,0', 'B,1,0'),
 ('1,0', 'B,1,0'),
 ('0,1', 'B,1,1'),
 ('1,1', 'B,1,1')]

reduceByKey() will perform the merging locally on each mapper before sending results to a reducer, similarly to a “combiner” in MapReduce. The output will be the multiplications which need to occur and position for each multiplication. 

In [52]:
reduced_numbers = mappings.reduceByKey(lambda a,b:a+'#'+b)
reduced_numbers.collect()

[('1,1', 'A,0,3#A,1,4#B,0,0#B,1,1'),
 ('0,0', 'A,0,1#A,1,2#B,0,1#B,1,0'),
 ('0,1', 'A,0,1#A,1,2#B,0,0#B,1,1'),
 ('1,0', 'A,0,3#A,1,4#B,0,1#B,1,0')]

Consider the first value ('1,1', 'A,0,3#A,1,4#B,0,0#B,1,1'). Here, (1,1) is the position of the output value from the multiplication which is given by 'A,0,3#A,1,4#B,0,0#B,1,1'. A,0,3 should be multiplied with B,0,0 and A,1,4 should be multiplied with B,1,1. Now, apply the operation using map_func2(). This will compute the value and map() will return a new RDD by applying the function to each element of the input RDD (reduced_numbers).

In [53]:
matrix = reduced_numbers.map(lambda x : (x[0],map_func2(x[1])))
matrix.collect()

[('1,1', '4'), ('0,0', '1'), ('0,1', '2'), ('1,0', '3')]

In [54]:
#Repeat for matrix_2.txt, dimenions of A is 2x3 and B is 3x2
m = 2
n = 3
p = 2

sc = SparkContext.getOrCreate();
numbers = sc.textFile("matrix_2.txt").flatMap(lambda line:line.split("\n"))
numbers.collect()

['A,0,0,1',
 'A,0,1,2',
 'A,0,2,-1',
 'A,1,0,2',
 'A,1,1,0',
 'A,1,2,1',
 'B,0,0,3',
 'B,0,1,1',
 'B,1,0,0',
 'B,1,1,-1',
 'B,2,0,-2',
 'B,2,1,3']

In [55]:
mappings = numbers.flatMap(map_func)
mappings.collect()

[('0,0', 'A,0,1'),
 ('0,1', 'A,0,1'),
 ('0,0', 'A,1,2'),
 ('0,1', 'A,1,2'),
 ('0,0', 'A,2,-1'),
 ('0,1', 'A,2,-1'),
 ('1,0', 'A,0,2'),
 ('1,1', 'A,0,2'),
 ('1,0', 'A,1,0'),
 ('1,1', 'A,1,0'),
 ('1,0', 'A,2,1'),
 ('1,1', 'A,2,1'),
 ('0,0', 'B,0,3'),
 ('1,0', 'B,0,3'),
 ('0,1', 'B,0,1'),
 ('1,1', 'B,0,1'),
 ('0,0', 'B,1,0'),
 ('1,0', 'B,1,0'),
 ('0,1', 'B,1,-1'),
 ('1,1', 'B,1,-1'),
 ('0,0', 'B,2,-2'),
 ('1,0', 'B,2,-2'),
 ('0,1', 'B,2,3'),
 ('1,1', 'B,2,3')]

In [56]:
reduced_numbers = mappings.reduceByKey(lambda a,b:a+'#'+b)
reduced_numbers.collect()

[('1,1', 'A,0,2#A,1,0#A,2,1#B,0,1#B,1,-1#B,2,3'),
 ('0,0', 'A,0,1#A,1,2#A,2,-1#B,0,3#B,1,0#B,2,-2'),
 ('0,1', 'A,0,1#A,1,2#A,2,-1#B,0,1#B,1,-1#B,2,3'),
 ('1,0', 'A,0,2#A,1,0#A,2,1#B,0,3#B,1,0#B,2,-2')]

In [57]:
matrix = reduced_numbers.map(lambda x : (x[0],map_func2(x[1])))
matrix.collect()

[('1,1', '5'), ('0,0', '5'), ('0,1', '-4'), ('1,0', '4')]